In [1]:
import pandas as pd
import csv

In [20]:
global uploaded
uploaded = "bpi2012.csv"

In [106]:
# import utility.run
import os
import sys
from enum import Enum
from keras import backend
import csv
import unicodecsv
import numpy as np
import copy
from abc import ABC, abstractmethod
# from tensorflow import keras 

# from utility.enums import DataGenerationPattern, Processor, RnnType

# Dataoperation
# from .enums import DataType as dt
# from .enums import DataClass as dc
# from .enums import FeatureType as ft

In [107]:
# # from utility.enums import Processor

# def Configure(args):    
#     import keras as keras
#     print("keras version: " + keras.__version__)

#     backendname = backend.backend()
#     if backendname == 'cntk':
#         __configure_CNTK(args)
#     elif backendname == 'tensorflow':
#         __configure_tensorflow(args)
#     elif backendname == 'theano':
#         __configure_Theano(args)
#     else:
#         raise ValueError("unable to detect backend for configuration")

# # def Clean_Session():
# #     backendname = backend.backend()
# #     if backendname == 'cntk':
# #         __clean_session_cntk()
# #     elif backendname == 'tensorflow':
# #         __clean_session_tensorflow()
# #     elif backendname == 'theano':
# #         __clean_session_theano()
# #     else:
# #         raise ValueError("unable to detect backend for cleanup")
# #     # cleanup
# #     from keras import backend as K
# #     K.clear_session()

# # def __configure_Theano(args):
# #     print("theano configured")

# # def __configure_CNTK(args):
# #     print("cntk configured")

# # def __configure_tensorflow(args):
# #     import tensorflow as tf
# #     print("tensorflow version: " + tf.__version__)
# #     # tf.logging.set_verbosity(tf.logging.INFO)
# #     # commented out
# #     Clean_Session()

#     if(args['processor'] != Processor.GPU):
#         import os
#         os.environ["CUDA_VISIBLE_DEVICES"]="-1"
#     if(args['processor'] == Processor.TPU):
#         # setup tpu worker for google colab (colab.research.google.com)
#         tpu_addr = os.environ.get('COLAB_TPU_ADDR')
#         if(tpu_addr is not None):
#             args['TPU_WORKER'] = 'grpc://' + tpu_addr
#             print("tpu worker: {}".format(args['TPU_WORKER']))
#         else:
#             args['processor'] = Processor.CPU
#             print("no tpu worker found, falling back to cpu")
#     elif(args['processor'] == Processor.GPU):
# #         from keras.backend.tensorflow_backend import set_session
#         config = tf.ConfigProto()
#         config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#         sess = tf.Session(config=config)
#         set_session(sess)  # set this TensorFlow session as the default session for Keras
#         print("multi process gpu usage: enabled")
#     print("tensorflow configured")

# # def __clean_session_theano():
# #     print("theano session cleaned")

# # def __clean_session_cntk():
# #     print("cntk session cleaned")

# # def __clean_session_tensorflow():
# #     import tensorflow as tf
# # #     tf.keras.backend.clear_session()
# #     print("tensorflow session cleaned")

In [108]:
# Dataoperations


minutes_in_day = 24 * 60
days_in_week = 7

def ReadInData(file, row_structure):
    csvfile = open(file, 'r')
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None)  # skip the headers
    lastcase = ''
    firstLine = True
    totalrows = 0

    #create return array with size of len(row_structure) (columns)
    data = []
    intermediateData = []
    for i in range(len(row_structure)):
        data.append([])
        intermediateData.append([])

    for row in spamreader:
        totalrows = totalrows + 1
        if row[0] != lastcase:
            lastcase = row[0]
            if not firstLine:
                for i in range(len(row_structure)):
                    data[i].append(intermediateData[i])
            for i in range(len(row_structure)):
                intermediateData[i] = []
        for i in range(len(row_structure)):
            #cast by defined type
            if row_structure[i]['datatype'] == DataType.String:                
                intermediateData[i].append(row[row_structure[i]['columnindex']])
            elif row_structure[i]['datatype'] == DataType.Float:
                intermediateData[i].append(float(row[row_structure[i]['columnindex']]))
            elif row_structure[i]['datatype'] == DataType.Int:
                intermediateData[i].append(int(row[row_structure[i]['columnindex']]))
            else:
                raise ValueError("unknown datatype encountered")
        firstLine = False
    # add last case
    for i in range(len(row_structure)):
        data[i].append(intermediateData[i])
    print("read {} rows".format(totalrows))
    return data

def VerifyDatadefinition(datadefinition):
    # set default values
    for index, definition in enumerate(datadefinition):
        if 'datatype' not in definition:
            definition['datatype'] = DataType.String
            print("no value found for datadefinition.datatype at index {}, setting default value".format(index))
        if 'dataclass' not in definition:
            definition['dataclass'] = DataClass.Onehot
            print("no value found for datadefinition.dataclass at index {}, setting default value".format(index))
        if 'featuretype' not in definition:
            definition['featuretype'] = FeatureType.none
            print("no value found for datadefinition.featuretype at index {}, setting default value".format(index))
        if 'featureweight' not in definition:
            definition['featureweight'] = 1.0
            print("no value found for datadefinition.featureweight at index {}, setting default value".format(index))
        if 'columnindex' not in definition:
            raise ValueError("columnindex cannot be empty")

def CalculateFeatures(args):
    catvectorlen = 0
    num_features = 1 # default 1 because indexing variable
    args['feature_weights'] = []
    for index, definition in enumerate(args['rowstructure']):
        # append feature weight, if non provide add default
        if definition['featureweight']:
            args['feature_weights'].append(definition['featureweight'])
        else:
            args['feature_weights'].append(1.0)

        if definition['featuretype'] == FeatureType.Train:            
            if definition['datatype'] == DataType.String:
                if definition['dataclass'] == DataClass.Onehot:
                    catvectorlen += len(args['indices']['chars_indices'][index]) # onehot increases vector length by amount of classes (encoded as strings)
                elif definition['dataclass'] == DataClass.Multilabel:
                    catvectorlen += len(args['indices']['unique_chars_indices'][index]) # ml increases vector length by amount of classes (encoded as chars)
            elif definition['datatype'] == DataType.Float:
                if definition['dataclass'] == DataClass.Periodic:
                    num_features += 2 # periodic values increase by 2 (sin/cos)
                else:
                    num_features += 1 # every numeric value increases by 1
            elif definition['datatype'] == DataType.Int:
                if definition['dataclass'] == DataClass.Periodic:
                    num_features += 2 # periodic values increase by 2 (sin/cos)
                else:
                    num_features += 1 # every numeric value increases by 1            

    args['catvectorlen'] = catvectorlen
    print('category vectors length:', args['catvectorlen'])
    args['num_features'] = catvectorlen + num_features
    print('num features: {}'.format(args['num_features']))
    return

def AppendEOL(data):
    outdata = []
    for i in range(len(data)):
        if isinstance(data[i][0], float) or isinstance(data[i][0], int):
            outdata.append(list(map(lambda x: x + [0], data[i])))
        else:
            outdata.append(list(map(lambda x: x + ['!'],data[i])))
    return outdata

def TruncateSequences(indata, maxlength):
    outdata = []
    for i in range(len(indata)):
        outdata.append([])
    for i in range(len(indata[0])):
        if(len(indata[0][i]) <= maxlength):
            for j in range(len(indata)):
                outdata[j].append(indata[j][i])
    print("sequences truncated to {}".format(maxlength))
    return outdata

def CreateDivisors(data):
    # get divisors for normalization
    divisors = []
    for i in range(len(data)):
        if isinstance(data[i][0][0], float) or isinstance(data[i][0][0], int):
            divisors.append(np.mean([item for sublist in data[i] for item in sublist]))
        else:
            divisors.append("null")
        print('divisor{0}: {1}'.format(i, divisors[i]))
    return divisors

def CreateOffsets(data):
    offsets = []
    for i in range(len(data)):
        if isinstance(data[i][0][0], float) or isinstance(data[i][0][0], int):
            # normalize to: 0 to x
            offset = np.min([item for sublist in data[i] for item in sublist])
            if offset < 0:
                datatemp = []
                for seq in data[i]:
                    datatempint = []
                    for elem in seq:
                        datatempint.append(elem - offset)
                    datatemp.append(datatempint)
                data[i] = datatemp
                offsets.append(offset)
            else:
                offsets.append(0)
        else:
            offsets.append("null")
        print('offset{0}: {1}'.format(i, offsets[i]))
    return offsets

def CreateDictionaries(data, rowstructure):    
    resultDict = {
        "chars" : [], # corpus
        "unique_chars" : [], # corpus elements from multilabel classification
        "target_chars" : [], # corpus including eol
        "target_unique_chars" : [], # corpus elements from multilabel classification including eol

        "chars_indices" : [], # chars - index mapping
        "unique_chars_indices" : [], # unique chars - index mapping
        "target_chars_indices" : [], # target chars - index mapping
        "target_unique_chars_indices" : [],

        "indices_chars" : [], # index - chars mapping
        "indices_target_chars" : [], # index - target chars mapping
        "indices_unique_chars" : [], # index - unique chars mapping
        "indices_target_unique_chars" : []
    }

    for i in range(len(data)): 
        if isinstance(data[i][0][0], str) and (rowstructure[i]['dataclass'] == DataClass.Onehot or rowstructure[i]['dataclass'] == DataClass.Multilabel):
            # get chars
            buffer = map(lambda x : set(x),data[i])
            buffer = list(set().union(*buffer))
            buffer.sort()
            resultDict["target_chars"].append(copy.copy(buffer)) # fill target_chars
            buffer.remove('!')
            resultDict["chars"].append(buffer) # fill chars
            print('total chars: {}, target chars: {}'.format(len(resultDict["chars"][i]), len(resultDict["target_chars"][i])))
            print('characters: ', resultDict["chars"][i])

            #get unique chars
            buffer = [l for word in resultDict["chars"][i] for l in word]
            buffer.append('!')
            buffer = list(set(buffer))
            buffer.sort()
            resultDict["target_unique_chars"].append(copy.copy(buffer)) # fill target_unique_chars
            buffer.remove('!')
            resultDict["unique_chars"].append(buffer) # fill unique_chars
            print('unique characters: ', resultDict["unique_chars"][i])

            # map index dictionaries
            resultDict["chars_indices"].append(dict((c, i) for i, c in enumerate(resultDict["chars"][i]))) #dictionary<key,value> with <char, index> where char is unique symbol for activity
            resultDict["unique_chars_indices"].append(dict((c, i) for i, c in enumerate(resultDict["unique_chars"][i])))
            resultDict["target_chars_indices"].append(dict((c, i) for i, c in enumerate(resultDict["target_chars"][i])))
            resultDict["target_unique_chars_indices"].append(dict((c, i) for i, c in enumerate(resultDict["target_unique_chars"][i])))
            resultDict["indices_chars"].append(dict((i, c) for i, c in enumerate(resultDict["chars"][i]))) #dictionary<key,value> with <index, char> where char is unique symbol for activity
            resultDict["indices_unique_chars"].append( dict((i, c) for i, c in enumerate(resultDict["unique_chars"][i])))
            resultDict["indices_target_chars"].append(dict((i, c) for i, c in enumerate(resultDict["target_chars"][i])))
            resultDict["indices_target_unique_chars"].append(dict((i, c) for i, c in enumerate(resultDict["target_unique_chars"][i])))
        else: # append empty dictionaries for non-string types
            resultDict["target_chars"].append([])
            resultDict["chars"].append([])
            resultDict["target_unique_chars"].append([])
            resultDict["unique_chars"].append([])
            resultDict["chars_indices"].append([]) 
            resultDict["unique_chars_indices"].append([])
            resultDict["target_chars_indices"].append([])
            resultDict["target_unique_chars_indices"].append([])
            resultDict["indices_chars"].append([])
            resultDict["indices_unique_chars"].append([])
            resultDict["indices_target_chars"].append([])
            resultDict["indices_target_unique_chars"].append([])
    return resultDict

def CreateSentences(data):
    sentences = []
    for i in range(len(data)):
        buffer = []
        for j in range(len(data[0])):
            for k in range(len(data[0][j])):
                if k == 0: # skip sequence with length = 1
                    continue
                buffer.append(data[i][j][0:k])
        sentences.append(buffer)
    return sentences

def CreateNgramsFromLabels(data, rowstructure, ngram_size):
    """ replaces string input labels with their ngram representation """

    if type(ngram_size) is not int:
        raise ValueError("ngram_size is not an integer")
    if ngram_size < 1:
        raise ValueError("ngram_size must be an integer >= 1")

    for i, column in enumerate(rowstructure):
        #only ngram string variables
        if column['datatype'] == DataType.String:
            #iterate through data
            for j in range(len(data[i])):
                # iterate through words in sentence
                newsentence = []
                for k in range(len(data[i][j])):
                    newword = []
                    for l in range(ngram_size - 1,-1,-1):
                        if  k-l >= 0:
                            newword.append(data[i][j][k-l])
                    newsentence.append(' '.join(newword))
                data[i][j] = newsentence
            print("created ngrams for column ", i)

In [109]:
# enums.py
class DataType(Enum):
    String = "string"
    Float = "float"
    Int = "int"

class DataClass(Enum):
    Onehot = "onehot"
    Multilabel = "multilabel"
    Numeric = "numeric"
    Periodic = "periodic"
    Integer = "integer"
    none = "none"

class RnnType(Enum):
    RNN = "rnn"
    LSTM = "lstm"
    GRU = "gru"

class Processor(Enum):
    CPU = "cpu"
    GPU = "gpu"
    TPU = "tpu"

class FeatureType(Enum):
    Train = "train"
    Target = "target"
    none = "none"

class DataGenerationPattern(Enum):
    Fit = "Fit" # classic approach (aka load all into memory)
    Generator = "Generator" # generator approach (e.g. fit_generator)

In [110]:
# exceptions.py
   
class Error(Exception):
    pass

class ConductorError(Error):
    """Exception raised for errors during execution. Handled via custom element to make parsing easier
    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message):
        self.message = message

In [111]:
# Generator.py

#TODO: make tpu compliant
#TODO: shuffle switch

def count_sentences(data):
    datasize = 0
    for i in range(0,len(data[0])):
        datasize += len(data[0][i]) - 1 # remove EOL character
    print("count generator with {} sequences and {} sentences".format(len(data[0]), datasize))
    return datasize

def GenerateGenerator(isTensorflow, data, args,shuffle=True):
    # factory function
    if isTensorflow == True:
        import tensorflow.keras as keras
    else:
        import keras as keras
    generator = __GenerateGenerator(keras.utils.Sequence, data, args, shuffle)
    return generator

def __GenerateGenerator(base, data, args, shuffle):
    class DataGenerator(base):
            'Generates data for Keras'
            def __init__(self, data, args, shuffle):
                'Initialization'
                self.data = data
                self.shuffle = shuffle
                self.datadefinition = args['datadefinition']
                self.args = args
                self.datasize = 0
                self.current_index = 0

                self.buffer = []
                self.outbuffer = []
                self.newbuffer = []

                #self.on_epoch_end()
                #self.__shuffle_data()

                for i in range(len(data)):
                    self.buffer.append([])

                for i in range(0,len(self.data[0])):
                    self.datasize += len(self.data[0][i]) - 1 # remove EOL character
                print("created generator with with {} sequences and {} sentences".format(len(self.data[0]), self.datasize))

                if self.shuffle:
                    self.__shuffle_data()

            def __len__(self):
                'Denotes the number of batches per epoch'
                return int(np.floor(self.datasize / self.args['batch_size']))

            def __getitem__(self, index):
                'Generate one batch of data'
                # fill buffer until threshold reached
                while len(self.buffer[0]) < self.args['batch_size']:
                    sequence = []
                    for i in range(len(self.data)):
                        sequence.append([])
                        sequence[i].append(self.data[i][self.current_index])      
                    sentences = CreateSentences(sequence)
                    self.current_index +=1 
                    if self.current_index >= len(data[0]):
                        self.current_index = 0
                    # put sentences into buffer
                    for i in range(len(self.data)):
                        for j in range(len(sentences[i])):
                            self.buffer[i].append(sentences[i][j])
                
                # if buffer has reached certain size, yield
                self.outbuffer = []
                self.newbuffer = []
                for i in range(len(self.buffer)):
                    self.outbuffer.append([])
                    self.newbuffer.append([])
                    for j in range(len(self.buffer[i])):
                        if j < self.args['batch_size']:
                            self.outbuffer[i].append(self.buffer[i][j])
                        else:
                            self.newbuffer[i].append(self.buffer[i][j])
                # remove used entries by setting a new buffer with remaining entries
                self.buffer = self.newbuffer
                matrix = self.datadefinition.CreateMatrices(self.outbuffer,self.args)

                return matrix['X'], matrix['y_t']        

            def on_epoch_end(self):
                'after each epoch: reset index and shuffle'
                self.current_index = 0
                if self.shuffle == True:
                    self.__shuffle_data()

            def __shuffle_data(self):
                self.data = ShuffleArray(self.data)
    return DataGenerator(data, args, shuffle)

In [112]:
# Generator.py

import numpy as np
import random
# from utility.dataoperations import CreateSentences
# from utility.regularization import ShuffleArray

#TODO: make tpu compliant
#TODO: shuffle switch

def count_sentences(data):
    datasize = 0
    for i in range(0,len(data[0])):
        datasize += len(data[0][i]) - 1 # remove EOL character
    print("count generator with {} sequences and {} sentences".format(len(data[0]), datasize))
    return datasize

def GenerateGenerator(isTensorflow, data, args,shuffle=True):
    # factory function
    if isTensorflow == True:
        import tensorflow.keras as keras
    else:
        import keras as keras
    generator = __GenerateGenerator(keras.utils.Sequence, data, args, shuffle)
    return generator

def __GenerateGenerator(base, data, args, shuffle):
    class DataGenerator(base):
            'Generates data for Keras'
            def __init__(self, data, args, shuffle):
                'Initialization'
                self.data = data
                self.shuffle = shuffle
                self.datadefinition = args['datadefinition']
                self.args = args
                self.datasize = 0
                self.current_index = 0

                self.buffer = []
                self.outbuffer = []
                self.newbuffer = []

                #self.on_epoch_end()
                #self.__shuffle_data()

                for i in range(len(data)):
                    self.buffer.append([])

                for i in range(0,len(self.data[0])):
                    self.datasize += len(self.data[0][i]) - 1 # remove EOL character
                print("created generator with with {} sequences and {} sentences".format(len(self.data[0]), self.datasize))

                if self.shuffle:
                    self.__shuffle_data()

            def __len__(self):
                'Denotes the number of batches per epoch'
                return int(np.floor(self.datasize / self.args['batch_size']))

            def __getitem__(self, index):
                'Generate one batch of data'
                # fill buffer until threshold reached
                while len(self.buffer[0]) < self.args['batch_size']:
                    sequence = []
                    for i in range(len(self.data)):
                        sequence.append([])
                        sequence[i].append(self.data[i][self.current_index])      
                    sentences = CreateSentences(sequence)
                    self.current_index +=1 
                    if self.current_index >= len(data[0]):
                        self.current_index = 0
                    # put sentences into buffer
                    for i in range(len(self.data)):
                        for j in range(len(sentences[i])):
                            self.buffer[i].append(sentences[i][j])
                
                # if buffer has reached certain size, yield
                self.outbuffer = []
                self.newbuffer = []
                for i in range(len(self.buffer)):
                    self.outbuffer.append([])
                    self.newbuffer.append([])
                    for j in range(len(self.buffer[i])):
                        if j < self.args['batch_size']:
                            self.outbuffer[i].append(self.buffer[i][j])
                        else:
                            self.newbuffer[i].append(self.buffer[i][j])
                # remove used entries by setting a new buffer with remaining entries
                self.buffer = self.newbuffer
                matrix = self.datadefinition.CreateMatrices(self.outbuffer,self.args)

                return matrix['X'], matrix['y_t']        

            def on_epoch_end(self):
                'after each epoch: reset index and shuffle'
                self.current_index = 0
                if self.shuffle == True:
                    self.__shuffle_data()

            def __shuffle_data(self):
                self.data = ShuffleArray(self.data)
    return DataGenerator(data, args, shuffle)


In [113]:
# models.py

def CreateModel(args):
    keras_impl = __getKerasImplementation(args['processor'])
    if(args['processor'] == Processor.TPU):
        print("imported tensorflow.keras API for model creation")
    else:
        print("imported keras API for model creation")

    if args['rnntype'] == RnnType.LSTM :
        if args['processor'] == Processor.GPU and args['cudnn'] == True:
            print("creating stateless cudnn lstm model")
            return __createCUDNN_LSTM_Stateless(keras_impl,args) 
        else:
            print("creating stateless lstm model")
            return __createLSTM_Stateless(keras_impl,args)  
    elif args['rnntype'] == RnnType.GRU:
        if args['processor'] == Processor.GPU and args['cudnn'] == True:
            print("creating stateless cudnn gru model")
            return __createCUDNN_GRU_Stateless(keras_impl,args) 
        else:
            print("creating stateless gru model")
            return __createGRU_Stateless(keras_impl,args)   
    elif args['rnntype'] == RnnType.RNN:
        print("creating stateless rnn model")
        return __createRNN_Stateless(keras_impl,args) 
    else:
        raise ValueError("unkown model type")

def CreateCallbacks(args):
    callbacks = []
    keras_impl = __getKerasImplementation(args['processor'])

    callbacks.append(keras_impl.callbacks.EarlyStopping(monitor='val_loss', patience=args['patience_earlystopping']))
    if(args['save_model'] == True):
        callbacks.append(keras_impl.callbacks.ModelCheckpoint(args['modelfilename'], monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto'))
    if(args['processor'] != Processor.TPU):
        callbacks.append(keras_impl.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=args['patience_reducelr'], verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0))
    callbacks.append(keras_impl.callbacks.CSVLogger('{}_epochlogs.epochlog'.format(args['running'])))
    if args['tensorboard'] == True:
        callsbacks.append(tensorboard_cb = TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True))
    return callbacks

def CreateOptimizer(keras_impl, args):
    if args['processor'] == Processor.TPU:
        import tensorflow as tf 
        return tf.contrib.opt.NadamOptimizer(learning_rate=args['learningrate'], beta1=0.9, beta2=0.999, epsilon=1e-08)
    else:
        return keras_impl.optimizers.Nadam(lr=args['learningrate'], beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=args['gradientclipvalue'])   

def __getKerasImplementation(processor):
    if(processor == Processor.TPU):
        import tensorflow
        from tensorflow.python import keras as keras_impl
    else:
        import keras as keras_impl
    return keras_impl

#lstm
def __createCUDNN_LSTM_Stateless(keras_impl,args):
#     model = keras_impl.models.Sequential()
    model = keras_impl.Sequential()
    if args['layers'] == 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNLSTM(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform'),input_shape=(args['maxlen'],args['num_features']))) 
        else:
            model.add(keras_impl.layers.CuDNNLSTM(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=False, kernel_initializer='glorot_uniform'))        
    if args['layers'] > 1:
        if args['bidirectional'] == True:            
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNLSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform'),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.CuDNNLSTM(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=True, kernel_initializer='glorot_uniform'))
        for i in range(args['layers'] - 1):
            model.add(keras_impl.layers.BatchNormalization())
            if i == args['layers'] - 2:
                if args['bidirectional'] == True:
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNLSTM(args['neurons'], return_sequences=False, kernel_initializer='glorot_uniform'))) 
                else:
                    model.add(keras_impl.layers.CuDNNLSTM(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform'))                  
            else:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNLSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform')))
                else:
                    model.add(keras_impl.layers.CuDNNLSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform'))
    model.add(keras_impl.layers.BatchNormalization())
    model.add(keras_impl.layers.Dense(1, kernel_initializer='glorot_uniform', name='time_output'))

    opt = CreateOptimizer(keras_impl, args)
    model.compile(loss={'time_output':'mae'}, optimizer=opt)
    return model

def __createLSTM_Stateless(keras_impl,args):
#     model = keras_impl.models.Sequential()
    model = keras_impl.Sequential()

    if args['layers'] == 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.LSTM(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.LSTM(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))
    if args['layers'] > 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.LSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.LSTM(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
        for i in range(args['layers'] - 1):
            model.add(keras_impl.layers.BatchNormalization())
            if i == args['layers'] - 2:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.LSTM(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.LSTM(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))                
            else:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.LSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.LSTM(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
    model.add(keras_impl.layers.BatchNormalization())
    model.add(keras_impl.layers.Dense(1, kernel_initializer='glorot_uniform', name='time_output'))

    opt = CreateOptimizer(keras_impl, args)   
    model.compile(loss={'time_output':'mae'}, optimizer=opt)
    return model

#gru
def __createCUDNN_GRU_Stateless(keras_impl,args):
#     model = keras_impl.models.Sequential()
    model = keras_impl.Sequential()


    if args['layers'] == 1:
        if args['bidirectional'] == True:            
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform'),input_shape=(args['maxlen'],args['num_features'])))
        else:
             model.add(keras_impl.layers.CuDNNGRU(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=False, kernel_initializer='glorot_uniform'))       
    if args['layers'] > 1:
        if args['bidirectional'] == True:            
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform'),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.CuDNNGRU(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=True, kernel_initializer='glorot_uniform'))
        for i in range(args['layers'] - 1):
            model.add(keras_impl.layers.BatchNormalization())
            if i == args['layers'] - 2:
                if args['bidirectional'] == True:            
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform')))
                else:
                    model.add(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform'))                  
            else:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform')))
                else:
                    model.add(keras_impl.layers.CuDNNGRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform'))
    model.add(keras_impl.layers.BatchNormalization())
    model.add(keras_impl.layers.Dense(1, kernel_initializer='glorot_uniform', name='time_output'))

    opt = CreateOptimizer(keras_impl, args)
    model.compile(loss={'time_output':'mae'}, optimizer=opt)
    return model

def __createGRU_Stateless(keras_impl,args):
#     model = keras_impl.models.Sequential()
    model = keras_impl.Sequential()
    if args['layers'] == 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.GRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.GRU(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))        
    if args['layers'] > 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.GRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.GRU(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
        for i in range(args['layers'] - 1):
            model.add(keras_impl.layers.BatchNormalization())
            if i == args['layers'] - 2:
                if args['bidirectional'] == True:
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.GRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.GRU(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))                
            else:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.GRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.GRU(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
    model.add(keras_impl.layers.BatchNormalization())
    model.add(keras_impl.layers.Dense(1, kernel_initializer='glorot_uniform', name='time_output'))

    opt = CreateOptimizer(keras_impl, args)
    model.compile(loss={'time_output':'mae'}, optimizer=opt)
    return model

#rnn (vanilla)
def __createRNN_Stateless(keras_impl,args):
#     model = keras_impl.models.Sequential()
    model = keras_impl.Sequential()
    if args['layers'] == 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.SimpleRNN(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))        
    if args['layers'] > 1:
        if args['bidirectional'] == True:
            model.add(keras_impl.layers.Bidirectional(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']),input_shape=(args['maxlen'],args['num_features'])))
        else:
            model.add(keras_impl.layers.SimpleRNN(args['neurons'],input_shape=(args['maxlen'],args['num_features']),return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
        for i in range(args['layers'] - 1):
            model.add(keras_impl.layers.BatchNormalization())
            if i == args['layers'] - 2:
                if args['bidirectional'] == True:
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=False, kernel_initializer='glorot_uniform', dropout=args['dropout']))                
            else:
                if args['bidirectional'] == True: 
                    model.add(keras_impl.layers.Bidirectional(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout'])))
                else:
                    model.add(keras_impl.layers.SimpleRNN(args['neurons'],return_sequences=True, kernel_initializer='glorot_uniform', dropout=args['dropout']))
    model.add(keras_impl.layers.BatchNormalization())
    model.add(keras_impl.layers.Dense(1, kernel_initializer='glorot_uniform', name='time_output'))

    opt = CreateOptimizer(keras_impl, args)
    model.compile(loss={'time_output':'mae'}, optimizer=opt)
    return model

In [114]:
def Parse_Args(**kwargs):
    outArgs = {}

    # default values if not defined
    outArgs['eventlog'] = ""
    outArgs['running'] = 0
    outArgs['bagging'] = True
    outArgs['bagging_putback'] = True
    outArgs['bagging_size'] = 0.8
    outArgs['traindata_split'] = 1
    outArgs['traindata_index'] = 0
    outArgs['traindata_duplicate'] = 0
    outArgs['traindata_shuffle'] = False
    outArgs['validationdata_split'] = 0.2
    outArgs['testdata_split'] = 0.3333
    outArgs['max_sequencelength'] = 3000
    outArgs['batch_size'] = 16
    outArgs['neurons'] = 100
    outArgs['layers'] = 2
    outArgs['dropout'] = 0.1
    outArgs['max_epochs'] = 500
    outArgs['learningrate'] = 0.002
    outArgs['patience_earlystopping'] = 40
    outArgs['patience_reducelr'] = 10
    outArgs['gradientclipvalue'] = 3
    outArgs['processor'] = Processor.CPU
    outArgs['verbose'] = False
    outArgs['rnntype'] = RnnType.LSTM
    outArgs['bidirectional'] = False
    outArgs['cudnn'] = False
    outArgs['tensorboard'] = False
    outArgs['save_model'] = False
    outArgs['datageneration_pattern'] = DataGenerationPattern.Fit
    outArgs['target_host'] = ''

    # copy in data
    for key,value in kwargs.items():
        outArgs[key] = value
        print('{} defined: {}'.format(key,value)) 

    # default values for specific switches
    if outArgs['processor'] == Processor.TPU:
        outArgs['save_model'] = False
        print('processor = tpu; setting save_model to false, because model_checkpoint slows training down for tpu due to weight copying')

    return outArgs

In [115]:
# Regularization.py


def ShuffleArray(array):
    random_state = numpy.random.get_state()
    intermediate = []
    for i in range(len(array)):
        numpy.random.shuffle(array[i])
        shuffledArray = array[i]
        # reset state to before shuffle, to ensure same shuffling result https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
        numpy.random.set_state(random_state) 
        intermediate.append(shuffledArray)
    return intermediate

def BagArray(bagging_size, array, putback):
    intermediate = []
    if putback == False and bagging_size >= 1:
        bagging_size = 1
    indices = list(range(0, int(round(bagging_size * len(array[0])))))
    for i in range(len(array)):
        intermediate.append([])
    if putback == True:
        for i in range(int(round(bagging_size * len(array[0])))):
            num = random.randint(0, len(array[0]) - 1)
            for j in range(len(array)):
                intermediate[j].append(array[j][num])
    elif putback == False:
        narray = ShuffleArray(array)
        for i in indices:
            for j in range(len(narray)):
                intermediate[j].append(narray[j][i])
    return intermediate

def DuplicateData(duplicate_size, array):
    intermediate = []
    # create empty array
    for i in range(len(array)):
        intermediate.append([])
    # copy array
    for i in range(len(array[0])):
        for j in range(len(array)):
            intermediate[j].append(array[j][i])
    # add additional duplicates
    number_of_dupes = int(round(duplicate_size * len(array[0])))
    if number_of_dupes > 0:     
        # iterate through entire set (number_of_dupes > 1)
        for i in range(int(number_of_dupes / len(array[0]))):
            for j in range(len(array[0])):
                for k in range(len(array)):
                    intermediate[k].append(array[k][j])              
        # iterate through part of set (modulo)        
        for i in range(number_of_dupes % len(array[0])): 
            for j in range(len(array)):
                intermediate[j].append(array[j][i])

    return intermediate

In [121]:
# Run.py

from __future__ import print_function, division
from collections import Counter
import unicodecsv
import numpy as np
import random
import sys
import os
from os.path import basename
import copy
import time
import shutil
from datetime import datetime
from math import log

# custom imports
# import utility.dataoperations as dataoperations
# import utility.models as models
# import utility.regularization as regularization
# import utility.preprocessing as preprocessing
# import utility.configuration as configuration
# import utility.generator as generator
# import utility.exceptions as exceptions
# import utility.send_results as send_results
# from utility.enums import Processor, DataGenerationPattern


def Train_And_Evaluate(**kwargs):
    try:
        # args = preprocessing.Parse_Args(**kwargs)
        args = Parse_Args(**kwargs)

        # get dataset specific parameters
        datadefinition = args['datadefinition']
        args['rowstructure'] = datadefinition.GetRowstructure()
        if args['eventlog'] == "": # get dataset defined in rowstructure if not explicitly supplied
            args['eventlog'] = datadefinition.GetDataset()

        #setup (e.g. backend specifics)
        # configuration.Configure(args)
#         Configure(args)
        
        args = __Preprocessing(args)
        model = __Train_Model(args)
        __Evaluate_Model(args, model)
    except Exception as ex:
        # catch all, throw one exception for parsing
        # raise exceptions.ConductorError(ex)
        raise ConductorError(ex)

def __ReadData(indata, args):
    #read from csv file
#     data = dataoperations.ReadInData(args[indata],args['rowstructure'])
    data = ReadInData(args[indata],args['rowstructure'])

    # remove sequences longer than maxseqlength
#     data = dataoperations.TruncateSequences(data,args['max_sequencelength'])
    data = TruncateSequences(data,args['max_sequencelength'])
    return data

def __Preprocessing(args): 
    # clean datadefinion
#     dataoperations.VerifyDatadefinition(args['rowstructure'])
    VerifyDatadefinition(args['rowstructure'])

    #define result filenames
    args['modelfilename'] = '{}_model.h5'.format(args['running'])
    args['testresultsfilename'] = '{}_results.csv'.format(args['running'])

    # read from csv file
    data = __ReadData("eventlog",args)

    # offset data (if < 0) and create divisors
    args['offsets'] = CreateOffsets(data)
    args['divisors'] = CreateDivisors(data)

    # append EOL characters
    data = AppendEOL(data)
    args['maxlen'] = max(map(lambda x: len(x),data[0])) - 1 # minus eol
    print('maxlen {} '.format(args['maxlen']))

    # extract corpus
    args['indices'] = CreateDictionaries(data,args['rowstructure'])

    # calc feature lengths
    CalculateFeatures(args)

    # generate folds
    intermediate_fold_data = []
    args['traindata'] = []
    args['validationdata'] = []
    args['testdata'] = []
    # first round of folds
    split_index = int(len(data[0]) - (len(data[0]) * args['testdata_split']))
    for i in range(len(data)):
        intermediate_fold_data.append(data[i][:split_index])
        args['testdata'].append(data[i][split_index:])
    # second round of folds
    split_index = int(len(intermediate_fold_data[0]) - (len(intermediate_fold_data[0])*args['validationdata_split']))
    for i in range(len(intermediate_fold_data)):
        args['traindata'].append(intermediate_fold_data[i][:split_index])
        args['validationdata'].append(intermediate_fold_data[i][split_index:])

    print('{} sequences in train data'.format(len(args['traindata'][i])))
    print('{} sequences in test data'.format(len(args['testdata'][i])))
    print('{} sequences in validation data'.format(len(args['validationdata'][i])))

    # split traindata into traindatasplit sets
    if args['traindata_split'] > 1:
        split_traindata = []
        elems_per_split = int(round(len(args['traindata'][0])/args['traindata_split']))
        for split in range(args['traindata_split']):
            split_traindata_array = []
            for i in range(len(args['traindata'])):
                split_traindata_array.append(args['traindata'][i][split*elems_per_split:(split + 1)*elems_per_split])
            split_traindata.append(split_traindata_array)
        args['traindata'] = split_traindata[args['traindata_index']]
        print('{} sequences in split train data'.format(len(args['traindata'][i])))

    # duplicate traindata
    if args['traindata_duplicate'] > 0:
        print('{} elements before duplicating'.format(len(args['traindata'][i])))
        args['traindata'] = regularization.DuplicateData(args['traindata_duplicate'],args['traindata'])
        print('{} elements after duplicating'.format(len(args['traindata'][i])))

    # bag traindata
    if args['bagging']:
        print('{} elements before bagging with putback {}'.format(len(args['traindata'][i]), args['bagging_putback']))
        args['traindata'] = regularization.BagArray(args['bagging_size'],args['traindata'], args['bagging_putback'])
        print('{} elements after bagging with putback {}'.format(len(args['traindata'][i]), args['bagging_putback']))

    # shuffle traindata
    if args['traindata_shuffle']:
        args['traindata'] =  regularization.ShuffleArray(args['traindata'])
        print('traindata shuffled')

    # generate sentences from training data 
    if args['datageneration_pattern'] == DataGenerationPattern.Fit:
        print('perform full in-memory sentence generation')
        args['train_sentences'] = CreateSentences(args['traindata'])
        args['validation_sentences'] = CreateSentences(args['validationdata'])
        print('train_sentences:', len(args['train_sentences'][0]))
        print('validation_sentences:', len(args['validation_sentences'][0]))
    elif args['datageneration_pattern'] == DataGenerationPattern.Generator:
        isTensorflow = args['processor'] == Processor.TPU
        args['train_generator'] = generator.GenerateGenerator(isTensorflow, args['traindata'],args)
        args['validation_generator'] = generator.GenerateGenerator(isTensorflow, args['validationdata'],args,shuffle=False)
        print('created sentence generators')
    else:
        raise ValueError("unknown value for datageneration_pattern")
    return args

def __Train_Model(args):
    # start building input matrix
    if args['datageneration_pattern'] == DataGenerationPattern.Fit:
        print('Vectorization...')
        train_matrices = args['datadefinition'].CreateMatrices(args['train_sentences'],args)
        validation_matrices = args['datadefinition'].CreateMatrices(args['validation_sentences'],args)
        x_train = train_matrices['X']
        y_train = train_matrices['y_t']
        x_val = validation_matrices['X']
        y_val = validation_matrices['y_t']
    elif args['datageneration_pattern'] == DataGenerationPattern.Generator:
        print('Generators detected: Vectorization will be performed during runtime')
    else:
        raise ValueError("unknown value for datageneration_pattern")

    # build the model: 
    print('Build model...')
    model = CreateModel(args)
    callbacks =CreateCallbacks(args)

    model.summary()
    verbositylevel = 2
    if args['verbose']:
        verbositylevel = 1

    # use tpu calls if tpu is defined
    if(args['processor'] == Processor.TPU):
        #do stuff
        import tensorflow as tf
        model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(args['TPU_WORKER'])))
        args['batch_size'] = args['batch_size'] * 8 #give each tpu batch_size elements    
        print('tpu detected: adjusting batch_size to ',  args['batch_size'])    
    if(args['datageneration_pattern'] == DataGenerationPattern.Fit):
        model.fit(x_train, {'time_output':y_train}, validation_data=(x_val,y_val), verbose=verbositylevel, callbacks=callbacks, shuffle=True, batch_size=args['batch_size'], epochs=args['max_epochs'])
    elif(args['datageneration_pattern'] == DataGenerationPattern.Generator):
        model.fit_generator(generator=args['train_generator'],
            validation_data=args['validation_generator'],
            verbose=verbositylevel, 
            callbacks=callbacks, shuffle=True, epochs=args['max_epochs'])
    else:
        raise ValueError("no training data found")        
    return model

def __Evaluate_Model(args,model = None):
    #prediction:
    print('Load model for predictions...') 

    # save model file if not exists     
    if os.path.exists(args['modelfilename']) == False:
        model.save(args['modelfilename'])  
        print('Model file does not exist, saving...')
    
    if args['save_model'] == False and model is not None:
        # cannot load model because it's not saved, but it is supplied  
        if args['processor'] == Processor.TPU:
            args['processor'] = Processor.CPU # make predictions on a cpu based model
            model.save_weights('{}_modelweights.h5'.format(args['running']))
            import tensorflow as tf # if it runs on tpu: tensorflow
#             tf.keras.backend.clear_session()
            model = CreateModel(args)
            model.load_weights('{}_modelweights.h5'.format(args['running']))
            print('tpu detected: Model saved with last weights and converted to cpu model for cpu inference')
        else: 
            print('Model loaded from memory')
    elif args['save_model'] == True:
        # model loaded from file, by recreating the model and loading weights
        model = CreateModel(args)
        model.load_weights(args['modelfilename'])
        print('Model loaded from checkpoint')
    else:
        raise ValueError("no model for predictions supplied / found")        
    model.summary()

    #evaluate
    args['datadefinition'].MakePredictions(model,args)    
    # configuration.Clean_Session()
    configuration.Clean_Session()

    #send results
    if args['target_host'] != '':
        send_results.SendResultFiles(args, [args['modelfilename'], args['testresultsfilename']])

In [117]:
# send results


#http://docs.python-requests.org/en/latest/user/quickstart/#post-a-multipart-encoded-file

def SendResultFiles(args, files):
    url = args['target_host']
    for file in files:
        try:
            fin = open(file, 'rb')
            files = {'file': fin}
            headers = {'authtoken': 'thisisatoken'}
            r = requests.post(url, files=files, headers=headers)
            print(r.text)
        finally:
            fin.close()

In [118]:
class GenericDatadefinition(ABC): 
    """ generic implementation for the functions CreateMatrices and MakePredictions. needs to override GetDataset and GetRowstructure """

    @abstractmethod
    def GetDataset(self):
        raise NotImplementedError("not implemented")
     
    @abstractmethod
    def GetRowstructure(self):        
        raise NotImplementedError("not implemented")

    def CreateMatrices(self,sentences,args):
        divisors = args['divisors']
        indices = args['indices']
        weights = args['feature_weights']
        num_features = args['num_features']
        rowstructure = args['rowstructure']
        maxlen = args['maxlen']
        verbose = args['verbose']

        X = np.zeros((len(sentences[0]), maxlen, num_features), dtype=np.float32)
        y_t = np.zeros((len(sentences[0]),1), dtype=np.float32)

        for i in range(len(sentences[0])):
            leftpad = maxlen-len(sentences[0][i])
            
            # train matrix
            self.__EncodeMatrix(X,i,leftpad,rowstructure,sentences,divisors,indices,weights)
            
            # target matrix
            for k, struc in enumerate(rowstructure): 
                # target variables
                if struc['featuretype'] == FeatureType.Target:
                    # TODO: more than 1 target
                    y_t[i,0] = sentences[k][i][0]/divisors[k] #already offsetted

            if verbose and args['datageneration_pattern'] != DataGenerationPattern.Generator:
                sys.stdout.write("vectorized sequence {0} of {1}\r".format(i,len(sentences[0])))                    
                sys.stdout.flush()
        return {'X': X, 'y_t': y_t}

    def MakePredictions(self,model,args):
        #TODO
        print('Make predictions...')
        with open(args['testresultsfilename'], 'w', newline='') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow(["sequenceid","sequencelength","prefix","completion","prediction","gt_prediction","gt_planned","gt_instance","prefix_activities","suffix_activities"])
            sequenceid = 0
            print('sequences: {}'.format(len(args['testdata'][0])))    
            for i in range(len(args['testdata'][0])):
                sequencelength = len(args['testdata'][0][i]) - 1 #minus eol character
                ground_truth = args['testdata'][6][i][0] + args['offsets'][6] #undo offset
                ground_truth_processid = args['testdata'][8][i][0]
                for prefix_size in range(1,sequencelength):   
                    cropped_data = []
                    for a in range(len(args['testdata'])):
                        cropped_data.append(args['testdata'][a][i][:prefix_size])  
                    prefix_activities = args['testdata'][0][i][:prefix_size]
                    suffix_activities = args['testdata'][0][i][prefix_size:]
                    if '!' in prefix_activities:
                        break # make no prediction for this case, since this case has ended already 

                    # predict
                    y = model.predict(self.__EncodePrediction(cropped_data, args['rowstructure'], args['divisors'], args['indices'],args['feature_weights'], args['num_features'], args['catvectorlen'], args['maxlen']), verbose=0)
                    y_t = y[0][0]
                    y_t = (y_t * args['divisors'][6]) + args['offsets'][6] #undo offset and multiply by divisor to un-normalize
                    prediction = y_t

                    #output stuff (sequence, prefix)
                    output = []
                    output.append(sequenceid)
                    output.append(sequencelength)
                    output.append(prefix_size)
                    output.append(prefix_size / sequencelength)
                    output.append(prediction)
                    output.append(ground_truth)
                    output.append("")
                    output.append(ground_truth_processid)
                    output.append(' '.join(prefix_activities))
                    output.append(' '.join(suffix_activities))
                    spamwriter.writerow(output)    

                    # out if an prediction was requested for a productive system
                    #if productive_prediction:
                    #    print('PredictedBuffer={}'.format(y_t)) 

                sequenceid += 1
                if args['verbose']:
                    print("finished sequence {} of {}".format(sequenceid,len(args['testdata'][0])))
                #end sequence loop
        print('finished prediction')

    def __EncodePrediction(self, sentence, rowstructure, divisors, indices, weights,  num_features, catvectorlen, maxlen):
        X = np.zeros((1, maxlen, num_features), dtype=np.float32)
        leftpad = maxlen-len(sentence[0])
        self.__EncodeMatrix(X, 0, leftpad, rowstructure, sentence, divisors, indices, weights)
        return X
    
    def __EncodeMatrix(self, X, index, leftpad, rowstructure, sentences, divisors, indices, weights):
        for j in range (len(sentences[0][index])):
            catvectorpad = 0  
            # train matrix
            for k, struc in enumerate(rowstructure):                         
                if struc['featuretype'] == FeatureType.Train:
                    # manage strings/labels
                    if struc['datatype'] == DataType.String:
                        if struc['dataclass'] == DataClass.Onehot:
                            for c in indices["chars"][k]:
                                if c == sentences[k][index][j]:
                                    X[index, j+leftpad, catvectorpad + indices["chars_indices"][k][c]] = weights[k]
                            catvectorpad += len(indices["chars_indices"][k])
                        elif struc['dataclass'] == DataClass.Multilabel:
                            for c in indices["unique_chars"][k]:
                                if c in sentences[k][index][j]:
                                    X[index, j+leftpad, catvectorpad + indices["unique_chars_indices"][k][c]] = weights[k]
                            catvectorpad += len(indices["unique_chars_indices"][k])
                        else:
                            raise NotImplementedError("string dataclass not implemented")
                    # manage numerics
                    elif struc['datatype'] == DataType.Float:
                        if struc['dataclass'] == DataClass.Numeric:
                            X[index, j+leftpad, catvectorpad] = ((sentences[k][index][j]/divisors[k]) * weights[k]) 
                            catvectorpad += 1
                        elif struc['dataclass'] == DataClass.Periodic:
                            raise NotImplementedError("periodic dataclass not implemented")
                        else:
                            raise NotImplementedError("numeric dataclass not implemented")
                    # special case: no dataclasses for int; treat as numeric
                    elif struc['datatype'] == DataType.Int:                                
                        X[index, j+leftpad, catvectorpad] = ((sentences[k][index][j]/divisors[k]) * weights[k]) 
                        catvectorpad += 1
                    else:
                        raise NotImplementedError("datatype not implemented")
            X[index, j+leftpad, catvectorpad] = j + 1 #index  

In [119]:
import numpy as np
import csv
import sys
# from utility.enums import DataType as dt
# from utility.enums import DataClass as dc
# from utility.enums import FeatureType as ft
# from utility.enums import DataGenerationPattern, Processor, RnnType
# from datadefinitions.generic import GenericDatadefinition

class BPI2012(GenericDatadefinition):
    def GetDataset(self):
        return uploaded #"datasets/bpi2012.csv"

    def GetRowstructure(self):
        rowstructure = []
        rowstructure.append({'datatype':DataType.String, 'columnindex':2 , 'dataclass':DataClass.Onehot, 'featuretype':FeatureType.Train, 'featureweight':1.0}) #event label (onehot) 
        rowstructure.append({'datatype':DataType.String, 'columnindex':5, 'dataclass':DataClass.Onehot, 'featuretype':FeatureType.Train, 'featureweight':1.0 }) #resource   
        rowstructure.append({'datatype':DataType.Float, 'columnindex':3, 'dataclass':DataClass.Numeric, 'featuretype':FeatureType.Train, 'featureweight':1.0 }) #duration (always 0)
        rowstructure.append({'datatype':DataType.Float, 'columnindex':4, 'dataclass':DataClass.Numeric, 'featuretype':FeatureType.Train, 'featureweight':1.0 }) #timestamp    
    
        rowstructure.append({'datatype':DataType.String, 'columnindex':6, 'dataclass':DataClass.Onehot, 'featuretype':FeatureType.Target }) #t/f
        rowstructure.append({'datatype':DataType.String, 'columnindex':7, 'dataclass':DataClass.none, 'featuretype':FeatureType.none }) #caseid
        return rowstructure

    def CreateMatrices(self,sentences,args):
        divisors = args['divisors']
        indices = args['indices']
        weights = args['feature_weights']
        num_features = args['num_features']
        catvectorlen = args['catvectorlen']
        maxlen = args['maxlen']
        verbose = args['verbose']

        X = np.zeros((len(sentences[0]), maxlen, num_features), dtype=np.float32)
        y_t = np.zeros((len(sentences[0]),1), dtype=np.float32)

        for i in range (len(sentences[0])):
            leftpad = maxlen-len(sentences[0][i])
            for j in range (len(sentences[0][i])):      
                catvectorpad = 0    
                # set oh vector (event label)
                for c in indices["chars"][0]:
                    if c == sentences[0][i][j]:
                        X[i, j+leftpad, indices["chars_indices"][0][c]] = 1
                catvectorpad += len(indices["chars_indices"][0])
                # set oh vector (resources)
                for c in indices["chars"][1]:
                    if c == sentences[1][i][j]:
                        X[i, j+leftpad, catvectorpad + indices["chars_indices"][1][c]] = 1
                # set time vector
                X[i, j+leftpad, catvectorlen] = j + 1 #index
                X[i, j+leftpad, catvectorlen+1] = sentences[2][i][j]/divisors[2]
                X[i, j+leftpad, catvectorlen+2] = sentences[3][i][j]/divisors[3] 

            if sentences[4][i][0] == "False":
                y_value = 0
            else:
                y_value = 1
            y_t[i,0] = y_value
            if verbose and args['datageneration_pattern'] != DataGenerationPattern.Generator:
                sys.stdout.write("vectorized sequence {0} of {1}\r".format(i,len(sentences[0])))
                sys.stdout.flush()
        return {'X': X, 'y_t': y_t}

    def MakePredictions(self,model,args):
        print('Make predictions...')
        with open(args['testresultsfilename'], 'w', newline='') as csvfile:
            spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            spamwriter.writerow(["sequenceid","sequencelength","prefix","completion","prediction","gt_prediction","gt_planned","gt_instance","prefix_activities","suffix_activities"])
            sequenceid = 0
            print('sequences: {}'.format(len(args['testdata'][0])))    
            for i in range(len(args['testdata'][0])):
                sequencelength = len(args['testdata'][0][i]) - 1 #minus eol character
                ground_truth = args['testdata'][4][i][0]
                ground_truth_processid = args['testdata'][5][i][0]
                for prefix_size in range(1,sequencelength):   
                    cropped_data = []
                    for a in range(len(args['testdata'])):
                        cropped_data.append(args['testdata'][a][i][:prefix_size])  
                    prefix_activities = args['testdata'][0][i][:prefix_size]
                    suffix_activities = args['testdata'][0][i][prefix_size:]

                    # predict
                    y = model.predict(self.__EncodePrediction(cropped_data, args['divisors'], args['indices'], args['num_features'], args['catvectorlen'], args['maxlen']), verbose=0)
                    y_t = y[0][0]
                    prediction = y_t

                    #output stuff (sequence, prefix)
                    output = []
                    output.append(sequenceid)
                    output.append(sequencelength)
                    output.append(prefix_size)
                    output.append(prefix_size / sequencelength)
                    output.append(prediction)                    
                    output.append(ground_truth)
                    output.append(0.5) # binary prediction with [0...1]
                    output.append(ground_truth_processid)
                    output.append(' ')
                    output.append(' ')
                    #output.append(' '.join(prefix_activities))
                    #output.append(' '.join(suffix_activities))
                    spamwriter.writerow(output)    

                sequenceid += 1
                if args['verbose']:
                    print("finished sequence {} of {}".format(sequenceid,len(args['testdata'][0])))
                #end sequence loop
        print('finished prediction')

    def __EncodePrediction(self,sentence, divisors, indices,  num_features, catvectorlen, maxlen):
        X = np.zeros((1, maxlen, num_features), dtype=np.float32)
        leftpad = maxlen-len(sentence[0])
        for i in range(len(sentence[0])):
            catvectorpad = 0 
            # set oh vector
            for c in indices["chars"][0]:
                if c == sentence[0][i]:
                    X[0, i+leftpad, indices["chars_indices"][0][c]] = 1
            catvectorpad += len(indices["chars_indices"][0])
            # set ml vector
            for c in indices["chars"][1]:
                if c == sentence[1][i]:
                    X[0, i+leftpad, catvectorpad + indices["chars_indices"][1][c]] = 1
            X[0, i+leftpad, catvectorlen] = i + 1 #index
            X[0, i+leftpad, catvectorlen+1] = sentence[2][i]/divisors[2] 
            X[0, i+leftpad, catvectorlen+2] = sentence[3][i]/divisors[3]
        return X

In [120]:
# import utility.run
# import os
# import sys
# from utility.enums import DataGenerationPattern, Processor, RnnType

#check for env variable for cpu/gpu environment detection
processorType = os.environ.get("CONDUCTHOR_TYPE")
if processorType == "cpu":
    print("cpu environment detected")
elif processorType == "gpu":
    print("gpu environment detected")
else:
    print("unknown environment detected, defaulting to gpu")
    processorType = "cpu"

#import data definition
# import datadefinitions.bpi2012 as datadef
datadef = BPI2012()

# if len(sys.argv) > 1:
    # param = float(sys.argv[1])
# utility.run.
Train_And_Evaluate(
    #data
    eventlog= uploaded, #"datasets/bpi2012.csv",  # file to read in
    datadefinition=datadef,          # the data / matrix definitions
    running=0,                       # iterable / suffix
    datageneration_pattern = DataGenerationPattern.Fit, # Fit: uses the classical approach and loads everything into memory; Generator: uses the python generator pattern
    #regularization/dataset manipulation
    bagging=False,                   # perform bagging? 
    bagging_putback=True,            # (if bagging) elements get put back and can be drawn again 
    bagging_size=0.8,                # (if bagging) the split to bag train data
    traindata_split=1,               # split the train data into X pieces; default 1 (no split)
    traindata_split_index=0,         # the index of the split to pick; default 0
    traindata_duplicate=0,           # duplicates part of the train data to (virtually) inflate it; default 0 (no duplicates)
    traindata_shuffle=False,         # shuffle training data
    validationdata_split = 0.2,      # Validation data split from traindata (default 0.2)
    testdata_split = 0.3333,         # test data split from input data. takes data from the bottom of the dataset (default 0.333)
    max_sequencelength=150000,       # maximum allowed sequence length
    #framework/ann specifics
    batch_size=64,                   # batch size (set to 1 for stateful)
    rnntype=RnnType.LSTM,            # type of rnn cell to use: lstm, gru or rnn (vanilla)
    neurons=100,                     # neurons per layer
    dropout=0.1,                     # dropout per layer (not applicable to CuDNN)
    max_epochs = 500,                # maximum amount of epochs to run (uses early_stopping)
    layers=2,                        # layers for the rnn
    gradientclipvalue=3,             # value to clip the gradient to if it exceeds it
    patience_earlystopping=20,       # patience for early stopping
    patience_reducelr=20,            # patience for lr reduction
    processor=Processor.GPU,         # processor, cpu, gpu or tpu (gpu uses CUDNN based algorithms for lstm and gru if cudnn is set to true)
    cudnn=True,                      # (if GPU) utilizes special nVidia-CuDNN LSTM and GRU implementations
    stateful=False,                  # stateful or stateless model; default False
    #debug
    save_model=True,                # saves the model file each checkpoint (set to False for TPU usage)
    tensorboard=False,               # outputs tensorboard compatible eventlog into ./graph folder for visualization
    verbose=False)                  # prints out a lot of progress reports. do NOT use with cluster learning

unknown environment detected, defaulting to gpu
eventlog defined: bpi2012.csv
datadefinition defined: <__main__.BPI2012 object at 0x7fc56f4181f0>
running defined: 0
datageneration_pattern defined: DataGenerationPattern.Fit
bagging defined: False
bagging_putback defined: True
bagging_size defined: 0.8
traindata_split defined: 1
traindata_split_index defined: 0
traindata_duplicate defined: 0
traindata_shuffle defined: False
validationdata_split defined: 0.2
testdata_split defined: 0.3333
max_sequencelength defined: 150000
batch_size defined: 64
rnntype defined: RnnType.LSTM
neurons defined: 100
dropout defined: 0.1
max_epochs defined: 500
layers defined: 2
gradientclipvalue defined: 3
patience_earlystopping defined: 20
patience_reducelr defined: 20
processor defined: Processor.GPU
cudnn defined: True
stateful defined: False
save_model defined: True
tensorboard defined: False
verbose defined: False
no value found for datadefinition.featureweight at index 4, setting default value
no value 

ConductorError: module 'tensorflow' has no attribute 'get_default_graph'

In [36]:
# pip install tensorflow==2.2.0